In [1]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter

from market_dynamics import bs_delta, bs_generator, bs_call_price
from sim_data import SimulationData
from utils import call_payoff, stochastic_integral
from models import ControlNet
from train import train, test

In [2]:
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
comment = ""
log_dir = os.path.join('experiments', current_time + '_' + comment)
writer = SummaryWriter(log_dir)

In [3]:
n_simulations = 500 
initial_value = 100
sigma = 0.2
n_steps = 50
rf = 0
strike = 95
fc_dims = [16, 16]
LR = 0.01
EPOCHS = 20
p_train = 0.9

bs_params = {"n_simulations": n_simulations,
             "n_steps": n_steps,
             "initial_value": initial_value,
             "sigma": sigma,
             }

price_params = {"n_steps": n_steps,
                "initial_value": initial_value,
                "sigma": sigma,
                "rf": rf,
                "strike": strike,
                }

payoff_params = {"strike": strike}

n_train = int(p_train * n_simulations)
n_test = n_simulations - n_train

dt = SimulationData(bs_generator, bs_params, bs_call_price, price_params, call_payoff, payoff_params)
dt_train, dt_test = torch.utils.data.random_split(dt, [n_train, n_test])
data_loader_train = DataLoader(dt_train, batch_size=20, shuffle=True)
cn = ControlNet(n_steps, 1, fc_dims, 1)

optimizer = Adam(cn.parameters(), lr=LR)
criterion = torch.nn.MSELoss()
train(data_loader_train, cn, criterion , optimizer, EPOCHS, writer)

Epoch 0: : 0batch [00:00, ?batch/s]/root/projects/uni/fml/FML/venv/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 20])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 0: : 22batch [00:01, 16.04batch/s, loss=8.14]/root/projects/uni/fml/FML/venv/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 0: : 23batch [00:01, 15.79batch/s, loss=7.95]
Epoch 1: : 23batch [00:01, 18.77batch/s, loss=4.09]
Epoch 2: : 23batch [00:01, 19.61batch/s, loss=2.43]
Epoch 3: : 23batch [00:01,

In [4]:
def delta(x):
    return bs_delta(n_steps, x, sigma, rf, strike)

In [5]:
# Evaluation In-Sample

_ ,model_loss = test(data_loader_train, cn, criterion)

l = len(data_loader_train.dataset)
x, x_inc, payoff, price = data_loader_train.dataset[:l]

d_hedge = x.apply_(delta)
si = stochastic_integral(x_inc, d_hedge)
delta_loss = criterion(price + si, payoff)

print(model_loss)
print(delta_loss)

tensor(1.5849, grad_fn=<MseLossBackward>)
tensor(2.8651)


In [6]:
# Evaluation Out-Sample
data_loader_test = DataLoader(dt_test, batch_size=20, shuffle=True)

_ ,model_loss = test(data_loader_test, cn, criterion)

l = len(data_loader_test.dataset)
x, x_inc, payoff, price = data_loader_test.dataset[:l]

d_hedge = x.apply_(delta)
si = stochastic_integral(x_inc, d_hedge)
delta_loss = criterion(price + si, payoff)

print(model_loss)
print(delta_loss)

tensor(1.9028, grad_fn=<MseLossBackward>)
tensor(2.7848)
